In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [12]:
# Device
DEVICE = torch.device("cuda:0")
print('Device:', DEVICE)

# Hyperparameters
random_seed = 1
learning_rate = 0.0005
num_epochs = 30
batch_size = 128

# Architecture
num_features = 784 #3072
num_classes = 10

train_dataset = datasets.CIFAR10(root='./data', 
                                 train=True, 
                                 transform=transforms.ToTensor(),
                                 download=True)

test_dataset = datasets.CIFAR10(root='./data', 
                                train=False, 
                                transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)


Device: cuda:0
Files already downloaded and verified


In [13]:
class VGG16(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(VGG16, self).__init__()
        
        self.block_1 = nn.Sequential(
                nn.Conv2d(in_channels=3,
                          out_channels=64,
                          kernel_size=3,
                          stride=1,
                          # (1(32-1)- 32 + 3)/2 = 1
                          padding=1), 
                nn.ReLU(),
                nn.Conv2d(in_channels=64,
                          out_channels=64,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2,
                             stride=2)
        )
        
        self.block_2 = nn.Sequential(
                nn.Conv2d(in_channels=64,
                          out_channels=128,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=128,
                          out_channels=128,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2,
                             stride=2)
        )
        
        self.block_3 = nn.Sequential(        
                nn.Conv2d(in_channels=128,
                          out_channels=256,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),        
                nn.Conv2d(in_channels=256,
                          out_channels=256,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2,
                             stride=2)
        )
        
          
        self.block_4 = nn.Sequential(   
                nn.Conv2d(in_channels=256,
                          out_channels=512,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),        
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),        
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),            
                nn.MaxPool2d(kernel_size=2,
                             stride=2)
        )
        
        self.block_5 = nn.Sequential(
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),            
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),            
                nn.Conv2d(in_channels=512,
                          out_channels=512,
                          kernel_size=3,
                          stride=1,
                          padding=1),
                nn.ReLU(),    
                nn.MaxPool2d(kernel_size=2,
                             stride=2)             
        )
            
        self.classifier = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes),
        )
            
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
                nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    m.bias.detach().zero_()        
        
    def forward(self, x):

        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        x = self.block_4(x)
        x = self.block_5(x)

        x = x.view(x.size(0), -1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas
    
torch.manual_seed(random_seed)
model = VGG16(num_features=num_features,
              num_classes=num_classes)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [14]:
import keyboard
import time

def simulate_key_press(key):
    keyboard.press(key)
    time.sleep(0.1)  # Halte die Taste für 0.1 Sekunden gedrückt
    keyboard.release(key)


In [15]:
def compute_accuracy(model, data_loader):
    model.eval()
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100


time.sleep(1)
simulate_key_press('p')
start_time = time.time()
for epoch in range(num_epochs):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
                
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()             

    model.eval()
    
    print('Epoch: %03d/%03d' % (epoch+1, num_epochs))
    
total_training_time = (time.time() - start_time)

total_accuracy = compute_accuracy(model, test_loader).item()
print('Total Training Time: %.2f sec' % total_training_time)

print('Total Accuracy: %.2f Prozent' % total_accuracy)

time.sleep(1)
simulate_key_press('p')

Epoch: 001/030
Epoch: 002/030
Epoch: 003/030
Epoch: 004/030
Epoch: 005/030
Epoch: 006/030
Epoch: 007/030
Epoch: 008/030
Epoch: 009/030
Epoch: 010/030
Epoch: 011/030
Epoch: 012/030
Epoch: 013/030
Epoch: 014/030
Epoch: 015/030
Epoch: 016/030
Epoch: 017/030
Epoch: 018/030
Epoch: 019/030
Epoch: 020/030
Epoch: 021/030
Epoch: 022/030
Epoch: 023/030
Epoch: 024/030
Epoch: 025/030
Epoch: 026/030
Epoch: 027/030
Epoch: 028/030
Epoch: 029/030
Epoch: 030/030
Total Training Time: 406.73 sec
Total Accuracy: 79.62 Prozent


In [24]:
import csv
import os

#Zum loggen der Trainingslaufzeit und accuracy  des Models

def add_row_to_csv(filename, datensatz_name, frequency, voltage, runtime, accuracy):
    # Daten für die neue Zeile
    new_row = {'Datensatz': datensatz_name, 'Frequenz': frequency, 'voltage': voltage, 'Laufzeit': runtime, 'Accuracy': accuracy}
    
    # Überprüfe, ob die Datei existiert
    file_exists = os.path.isfile(filename)
    
    # Öffne die CSV-Datei im Modus 'a' (append), um Werte hinzuzufügen
    with open(filename, 'a', newline='') as csvfile:
        # Erstelle einen CSV-Writer, wenn die Datei neu erstellt wird
        writer = csv.DictWriter(csvfile, fieldnames=['Datensatz', 'Frequenz', 'voltage', 'Laufzeit', 'Accuracy'])
        
        # Schreibe die Kopfzeile, falls die Datei neu erstellt wurde
        if not file_exists:
            writer.writeheader()
        
        # Schreibe die neue Zeile in die CSV-Datei
        writer.writerow(new_row)

# Beispielaufruf der Funktion
filename = 'vgg16_cifar_spannung.csv'
datensatz_name = 'cifar'
frequency = '3600'
voltage = '1.1'
runtime = round(total_training_time, 2)
print(total_accuracy)
accuracy = round(total_accuracy, 2)

add_row_to_csv(filename, datensatz_name, frequency, voltage, runtime, accuracy)


79.6199951171875
